In [1]:
import os
import sys

# Changes the current path to find the source files
current_dir = os.getcwd()
while current_dir != os.path.abspath("../src"):
    os.chdir("..")
    current_dir = os.getcwd()
sys.path.append(os.path.abspath("Efficient-Computing/Detection/Gold-YOLO"))

OUTPUT_DIR = "../data/others/model_output"

folder_paths = [OUTPUT_DIR]

# Create the output directories if they doesn't exist
for folder_path in folder_paths:
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)

In [2]:
import albumentations as A
from albumentations.pytorch import ToTensorV2
from plot import get_bounding_boxes, create_bboxes_image
import cv2
import numpy as np
from layers import AMF_GD_YOLOv8
import matplotlib.pyplot as plt
from time import time
import random
import torch
import torch.nn as nn
from utils import Folders
from data_processing import ImageData
from tqdm.notebook import tqdm
from typing import Sequence, Dict, Any, Callable, List

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

### Run the YOLOv8 model

In [ ]:
from ultralytics.models.yolo import YOLO

# Load a pretrained YOLO model
model = YOLO("yolov8n.pt")

# Perform object detection on an image using the model
results = model("https://ultralytics.com/images/bus.jpg", save=True)

### Use the AMF GD YOLOv8 model

In [ ]:
class_names = {
    0: "Tree",
    1: "Tree_unsure",
    2: "Tree_disappeared",
    3: "Tree_replaced",
    4: "Tree_new",
}

class_indices = {value: key for key, value in class_names.items()}

model = AMF_GD_YOLOv8(3, 1, scale="s", class_names=class_names)

rgb_image_path = "../data/images/cropped/2023_122000_484000_RGB_hrl/2023_122000_484000_RGB_hrl_1_1.tif"
chm_image_path = (
    "../data/CHM_cropped_0p24/122000_484000/122000_484000_1_1_filtered_chm.tif"
)
annotations_path = "../data/annotations/cropped/2023_122000_484000_RGB_hrl/2023_122000_484000_RGB_hrl_1_1.json"

gt_bboxes, gt_classes = get_bounding_boxes(annotations_path)

output_name = "Model_output_test.jpg"
output_path = os.path.join(OUTPUT_DIR, output_name)

model.train()
output = model(rgb_image_path, chm_image_path, image_save_path=output_path)

In [ ]:
# from ultralytics.models.utils.loss import DETRLoss
# from layers import xywh2xyxy

# nc = 4
# loss_func = DETRLoss(nc=nc)

# # output[0][0][0, :, 0] = torch.Tensor([107.3209, 151.2296, 161.4145, 207.6499, 0.9837, 0.5472, 0.3150, 0.5183])

# pred_bboxes = output[0][0].unsqueeze(0).permute((0, 1, 3, 2))[:, :, :, :4]
# pred_scores = output[0][0].unsqueeze(0).permute((0, 1, 3, 2))[:, :, :, 4:]
# batch = {
#     "cls": torch.Tensor([class_indices[cls] for cls in classes]).to(dtype=torch.int64),
#     "bboxes": xywh2xyxy(torch.Tensor(bboxes)),
#     "gt_groups": [len(classes)],
# }

# prefect_pred_bboxes = batch["bboxes"].unsqueeze(0).unsqueeze(0)
# prefect_pred_scores = (
#     torch.where(
#         batch["cls"].unsqueeze(-1).repeat(1, nc)
#         == torch.arange(nc).unsqueeze(0).repeat(batch["cls"].shape[0], 1),
#         1.0,
#         -1.0,
#     )
#     .unsqueeze(0)
#     .unsqueeze(0)
# )

# loss = loss_func.forward(pred_bboxes, pred_scores, batch)
# print(f"{loss = }")

model.train()
for i in range(100):
    output = model(rgb_image_path, chm_image_path, image_save_path=output_path)
    loss = model.compute_loss(output[0], gt_bboxes, gt_classes)
    print(loss)
    (loss["loss_class"] + loss["loss_bbox"]).backward()

In [ ]:
num_epochs = 100

dataloader = ...
model = AMF_GD_YOLOv8(3, 1, scale="n", class_names=class_names)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-1)
scheduler = torch.optim.lr_scheduler.LambdaLR(
    optimizer, lambda i: 1 / np.sqrt(i + 2), last_epoch=-1, verbose=True
)

model.train()

for epoch in range(num_epochs):
    for inputs, targets in dataloader:
        inputs, targets = inputs.to(device), targets.to(device)

        rgb_image = ...
        chm_image = ...

        optimizer.zero_grad()

        output = model(rgb_image, chm_image)
        loss = model.compute_loss(output[0], gt_bboxes, gt_classes)
        total_loss = loss["loss_class"] + loss["loss_bbox"]
        total_loss.backward()

        optimizer.step()

    scheduler.step()
    print(f"Loss class: {loss["loss_class"]:.4f}, Loss bbox: {loss["loss_bbox"]:.4f}")

### How to use data augmentation with Albumentations

In [ ]:
class Pad(A.PadIfNeeded):
    def __init__(
        self,
        added_height: int | None = 64,
        added_width: int | None = 64,
        pad_height_divisor: int | None = None,
        pad_width_divisor: int | None = None,
        position: A.PadIfNeeded.PositionType | str = A.PadIfNeeded.PositionType.CENTER,
        border_mode: int = cv2.BORDER_REFLECT_101,
        value: float | Sequence[float] | None = None,
        mask_value: float | Sequence[float] | None = None,
        always_apply: bool = False,
        p: float = 1,
    ):
        super().__init__(
            0,
            0,
            pad_height_divisor,
            pad_width_divisor,
            position,
            border_mode,
            value,
            mask_value,
            always_apply,
            p,
        )
        self.added_height = added_height
        self.added_width = added_width

    def update_params(self, params: Dict[str, Any], **kwargs: Any) -> Dict[str, Any]:
        params = super().update_params(params, **kwargs)
        rows = params["rows"]
        cols = params["cols"]
        self.min_height = rows + self.added_height
        self.min_width = cols + self.added_width
        return super().update_params(params, **kwargs)

In [ ]:
crop_size = 640
distort_steps = 30
distort_limit = 0.2

bbox_params = A.BboxParams(
    format="coco", min_area=0, min_visibility=0.2, label_fields=["class_labels"]
)

transform_spatial = A.Compose(
    [
        A.RandomCrop(width=crop_size, height=crop_size, p=1.0),
        A.GridDistortion(
            num_steps=distort_steps,
            distort_limit=(-distort_limit, distort_limit),
            border_mode=cv2.BORDER_CONSTANT,
            normalized=True,
            p=0.5,
        ),
        A.HorizontalFlip(p=0.5),
        A.RandomRotate90(p=1.0),
        A.Perspective(interpolation=cv2.INTER_LINEAR, p=0.25),
    ],
    bbox_params=bbox_params,
)

transform_pixel = A.Compose(
    [
        A.RandomCrop(width=crop_size, height=crop_size, p=1.0),
        A.Sharpen(p=0.25),
        A.RingingOvershoot(p=0.5),
        A.RandomGamma(p=1.0),
        A.GaussianBlur(p=0.5),
        A.GaussNoise(p=0.5),
        A.FancyPCA(alpha=1.0, p=0.5),
        A.Emboss(p=0.5),
        A.Blur(p=0.5),
        A.RandomBrightnessContrast(p=1.0),
        A.ChannelDropout(p=0.25),
        A.CLAHE(clip_limit=2.0, p=0.25),
        # A.Normalize(p=1.0),
    ],
)

# image_path = "../data/images_cropped/2023_122000_484000_RGB_hrl/2023_122000_484000_RGB_hrl_2_3.tif"
# annotations_path = "../data/annotations_cropped/2023_122000_484000_RGB_hrl/2023_122000_484000_RGB_hrl_2_3.json"
image_path = "../data/images_full/2023_122000_484000_RGB_hrl.tif"
annotations_path = "../data/annotations_full/3_all_annotations.json"

t = time()

image = cv2.imread(image_path)
bboxes, labels = get_bounding_boxes(annotations_path)

print(f"Time elapsed: {round(time() - t, 3)}")

number_tests = 10

colors = {
    "Tree": (104, 201, 45),
    "Tree_unsure": (255, 215, 158),
    "Tree_disappeared": (158, 174, 255),
    "Tree_replaced": (255, 90, 82),
    "Tree_new": (251, 106, 225),
}

t = time()

for i in range(number_tests):
    transformed_spatial = transform_spatial(
        image=image, bboxes=bboxes, class_labels=labels
    )
    transformed_spatial_image = transformed_spatial["image"]
    transformed_bboxes = transformed_spatial["bboxes"]
    transformed_class_labels = transformed_spatial["class_labels"]
    transformed = transform_pixel(image=transformed_spatial_image)
    transformed_image = transformed["image"]

    output_name = f"Augmentation_test_{i}.tif"
    output_path = os.path.join(OUTPUT_DIR, output_name)
    cv2.imwrite(output_path, transformed_image)

    bboxes_image = create_bboxes_image(
        transformed_image,
        transformed_bboxes,
        labels=transformed_class_labels,
        colors_dict=colors,
        scores=np.random.rand((len(bboxes))),
    )

    output_name = f"Augmentation_test_{i}_bboxes.tif"
    output_path = os.path.join(OUTPUT_DIR, output_name)
    cv2.imwrite(output_path, bboxes_image)

print(f"Time elapsed: {round(time() - t, 3)}")

### PyTorch dataset

In [4]:
import os
from torch.utils.data import Dataset, DataLoader
from utils import get_file_base_name
from skimage import io


class TreeDataset(Dataset):
    """Tree dataset."""

    def __init__(
        self,
        annotations_folder_path: str,  # TODO: Change to take a list of files instead (to separate train and validation)
        rgb_folder_path: str,
        chm_folder_path: str,
        labels_to_index: Dict[str, int],
        transform_spatial: Callable | None = None,
        transform_pixel: Callable | None = None,
    ) -> None:
        self.annotations_list: List[str] = []
        self.bboxes: Dict[str, List[List[float]]] = {}
        self.labels: Dict[str, List[int]] = {}
        for file_name in os.listdir(annotations_folder_path):
            annotations_file_path = os.path.join(annotations_folder_path, file_name)
            base_name = get_file_base_name(annotations_file_path)
            self.annotations_list.append(base_name)
            bboxes, labels = get_bounding_boxes(annotations_file_path)
            self.bboxes[base_name] = [bbox.as_list() for bbox in bboxes]
            self.labels[base_name] = [labels_to_index[label] for label in labels]

        self.rgb_folder_path = rgb_folder_path
        self.chm_folder_path = chm_folder_path
        self.transform_spatial = transform_spatial
        self.transform_pixel = transform_pixel

    def __len__(self) -> int:
        # return len(self.landmarks_frame)
        return len(self.annotations_list)

    def __getitem__(self, idx):
        # if torch.is_tensor(idx):
        #     idx = idx.tolist()

        # Read the images
        base_name = self.annotations_list[idx]
        rgb_path = os.path.join(self.rgb_folder_path, f"{base_name}.tif")
        image_rgb = io.imread(rgb_path)
        chm_path = os.path.join(self.chm_folder_path, f"{base_name}.tif")
        image_chm = io.imread(chm_path)

        # Get bboxes and labels
        bboxes = self.bboxes[base_name]
        labels = self.labels[base_name]

        # Apply the spatial transform to the two images, bboxes and labels
        if self.transform_spatial is not None:
            transformed_spatial = transform_spatial(
                image=image_rgb, image_chm=image_chm, bboxes=bboxes, class_labels=labels
            )
            image_rgb = transformed_spatial["image"]
            image_chm = transformed_spatial["image_chm"]
            bboxes = transformed_spatial["bboxes"]
            labels = transformed_spatial["class_labels"]

        # Apply the pixel transform the to RGB image
        if transform_pixel is not None:
            transformed = transform_pixel(image=image_rgb)
            image_rgb = transformed["image"]

        to_tensor = ToTensorV2()

        sample = {
            "image_rgb": to_tensor(image=image_rgb),
            "image_chm": to_tensor(image=image_chm),
            "bboxes": torch.tensor(bboxes),
            "labels": torch.tensor(labels),
        }
        return sample


def custom_collate_fn(batch):
    # Initialize lists to hold the extracted components
    rgb_images = []
    chm_images = []
    bboxes = []
    labels = []
    indices = []

    last_idx = 0

    # Iterate through the batch
    for item in batch:
        # Extract the components from the dictionary
        rgb_image = item["image_rgb"]["image"]
        chm_image = item["image_chm"]["image"]
        bbox = item["bboxes"]
        label = item["labels"]

        # Append the extracted components to the lists
        rgb_images.append(rgb_image)
        chm_images.append(chm_image)
        bboxes.append(bbox)
        labels.append(label)

        next_idx = last_idx + len(bbox)
        indices.append([last_idx, next_idx])
        last_idx = next_idx

    # Convert the lists to tensors and stack them
    rgb_images = torch.stack(rgb_images, dim=0)
    chm_images = torch.stack(chm_images, dim=0)
    # bboxes = torch.stack(
    #     [torch.cat((bbox, -torch.ones(max_len - len(bbox), 4))) for bbox in bboxes],
    #     dim=0,
    # )
    # labels = torch.stack(
    #     [torch.cat((label, -torch.ones(max_len - len(label)))) for label in labels],
    #     dim=0,
    # )
    bboxes = torch.cat(bboxes).to(torch.float32)
    labels = torch.cat(labels)
    indices = torch.tensor(indices)

    batch = {
        "image_rgb": rgb_images,
        "image_chm": chm_images,
        "bboxes": bboxes,
        "labels": labels,
        "indices": indices,
    }

    return batch

In [5]:
from collections import defaultdict


class MetricMonitor:
    def __init__(self, float_precision=3):
        self.float_precision = float_precision
        self.reset()

    def reset(self):
        self.metrics = defaultdict(lambda: {"val": 0.0, "count": 0, "avg": 0.0})

    def update(self, metric_name: str, val: float, count: int = 1):
        metric = self.metrics[metric_name]

        metric["val"] += val
        metric["count"] += count
        metric["avg"] = metric["val"] / metric["count"]

    def __str__(self):
        return " | ".join(
            [
                f"{metric_name}: {metric["avg"]:.{self.float_precision}f}"
                for (metric_name, metric) in self.metrics.items()
            ]
        )


def perfect_preds(
    gt_bboxes: torch.Tensor, gt_classes: torch.Tensor, gt_indices: torch.Tensor
):
    extracted_bboxes = []
    extracted_classes = []
    for idx in gt_indices:
        start_idx, end_idx = idx
        slice_bboxes = gt_bboxes[start_idx:end_idx]
        extracted_bboxes.append(slice_bboxes)
        slice_classes = gt_classes[start_idx:end_idx].long()
        extracted_classes.append(slice_classes)
    scores = [
        20 * nn.functional.one_hot(cls, num_classes=5) - 0.5
        for cls in extracted_classes
    ]
    prefect_preds = [
        torch.cat((bboxes, classes), dim=1).permute((1, 0)).unsqueeze(0)
        for bboxes, classes in zip(extracted_bboxes, scores)
    ]
    perfect_preds = torch.cat(
        [
            torch.cat(
                (
                    pred,
                    torch.zeros(
                        (pred.shape[0], pred.shape[1], 8400 - pred.shape[2])
                    ).to(pred.device),
                ),
                dim=2,
            )
            for pred in prefect_preds
        ]
    )
    # print(f"{perfect_preds.shape = }")
    return perfect_preds


def train(
    train_loader: DataLoader,
    model: nn.Module,
    optimizer: torch.optim.Optimizer,
    epoch: int,
    device: torch.device,
):
    metric_monitor = MetricMonitor()
    model.train()
    stream = tqdm(train_loader)
    for i, data in enumerate(stream, start=1):
        # for name, module in model.named_modules():
        #     print(f"{name}: {next(iter(module.parameters()))[0][0]}")
        #     break
        image_rgb: torch.Tensor = data["image_rgb"]
        image_chm: torch.Tensor = data["image_chm"]
        gt_bboxes: torch.Tensor = data["bboxes"]
        gt_classes: torch.Tensor = data["labels"]
        gt_indices: torch.Tensor = data["indices"]

        image_rgb = image_rgb.to(device, non_blocking=True)
        image_chm = image_chm.to(device, non_blocking=True)
        gt_bboxes = gt_bboxes.to(device, non_blocking=True)
        gt_classes = gt_classes.to(device, non_blocking=True)
        gt_indices = gt_indices.to(device, non_blocking=True)

        output = model(image_rgb, image_chm)
        loss = model.compute_loss(
            output, gt_bboxes, gt_classes, gt_indices, bboxes_format="xyxy"
        )
        total_loss = loss["loss_class"] + loss["loss_bbox"]
        # total_loss = torch.sum(torch.tensor(list(loss.values()), dtype=torch.float32))

        batch_size = image_rgb.shape[0]
        metric_monitor.update("Loss", total_loss.item(), batch_size)

        optimizer.zero_grad()
        total_loss.backward()
        optimizer.step()

        stream.set_description(f"Epoch: {epoch}. Train.      {metric_monitor}")


def validate(
    val_loader: DataLoader, model: nn.Module, epoch: int, device: torch.device
):
    metric_monitor = MetricMonitor()
    model.eval()
    stream = tqdm(val_loader)
    with torch.no_grad():
        for i, data in enumerate(stream, start=1):
            image_rgb: torch.Tensor = data["image_rgb"]
            image_chm: torch.Tensor = data["image_chm"]
            gt_bboxes: torch.Tensor = data["bboxes"]
            gt_classes: torch.Tensor = data["labels"]
            gt_indices: torch.Tensor = data["indices"]

            image_rgb = image_rgb.to(device, non_blocking=True)
            image_chm = image_chm.to(device, non_blocking=True)
            gt_bboxes = gt_bboxes.to(device, non_blocking=True)
            gt_classes = gt_classes.to(device, non_blocking=True)
            gt_indices = gt_indices.to(device, non_blocking=True)

            output = model(image_rgb, image_chm)
            loss = model.compute_loss(
                output, gt_bboxes, gt_classes, gt_indices, bboxes_format="xyxy"
            )
            # prefect_preds = perfect_preds(gt_bboxes, gt_classes, gt_indices)
            # loss = model.compute_loss(
            #     prefect_preds, gt_bboxes, gt_classes, gt_indices, bboxes_format="xyxy"
            # )
            total_loss = loss["loss_class"] + loss["loss_bbox"]

            batch_size = image_rgb.shape[0]
            metric_monitor.update("Loss", total_loss.item(), batch_size)
            stream.set_description(f"Epoch: {epoch}. Validation. {metric_monitor}")


def train_and_validate(
    model: nn.Module,
    train_dataset: Dataset,
    val_dataset: Dataset,
    lr: float,
    epochs: int,
    batch_size: int,
    num_workers: int,
    device: torch.device,
) -> nn.Module:
    train_loader = DataLoader(
        train_dataset,
        batch_size=batch_size,
        shuffle=True,
        num_workers=num_workers,
        pin_memory=True,
        collate_fn=custom_collate_fn,
    )
    val_loader = DataLoader(
        val_dataset,
        batch_size=batch_size,
        shuffle=False,
        num_workers=num_workers,
        pin_memory=True,
        collate_fn=custom_collate_fn,
    )
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    scheduler = torch.optim.lr_scheduler.LambdaLR(
        optimizer, lambda i: 1 / np.sqrt(i + 2), last_epoch=-1, verbose=True
    )

    for epoch in range(1, epochs + 1):
        train(train_loader, model, optimizer, epoch, device)
        validate(val_loader, model, epoch, device)
        scheduler.step()
    return model

In [6]:
# import gc

# gc.collect()
# torch.cuda.empty_cache()
# with torch.no_grad():
#     torch.cuda.empty_cache()
# del model

# Model

class_names = {
    0: "Tree",
    1: "Tree_unsure",
    2: "Tree_disappeared",
    3: "Tree_replaced",
    4: "Tree_new",
}

class_indices = {value: key for key, value in class_names.items()}

model = AMF_GD_YOLOv8(3, 1, device=device, scale="n", class_names=class_names).to(
    device
)

# Transformations for data augmentation

crop_size = 640
distort_steps = 30
distort_limit = 0.2

transform_spatial = A.Compose(
    [
        A.RandomCrop(width=crop_size, height=crop_size, p=1.0),
        A.GridDistortion(
            num_steps=distort_steps,
            distort_limit=(-distort_limit, distort_limit),
            border_mode=cv2.BORDER_CONSTANT,
            normalized=True,
            p=0.5,
        ),
        A.HorizontalFlip(p=0.5),
        A.RandomRotate90(p=1.0),
        A.Perspective(interpolation=cv2.INTER_LINEAR, p=0.25),
    ],
    bbox_params=A.BboxParams(
        format="pascal_voc",
        min_area=0,
        min_visibility=0.2,
        label_fields=["class_labels"],
    ),
    additional_targets={"image_chm": "image"},
)

transform_pixel = A.Compose(
    [
        A.RandomCrop(width=crop_size, height=crop_size, p=1.0),
        A.Sharpen(p=0.25),
        A.RingingOvershoot(p=0.5),
        A.RandomGamma(p=1.0),
        A.GaussianBlur(p=0.5),
        A.GaussNoise(p=0.5),
        A.FancyPCA(alpha=1.0, p=0.5),
        A.Emboss(p=0.5),
        A.Blur(p=0.5),
        A.RandomBrightnessContrast(p=1.0),
        A.ChannelDropout(p=0.25),
        A.CLAHE(clip_limit=2.0, p=0.25),
        A.Normalize(p=1.0),
    ],
)

# Paths to the data

full_image_path = "../data/images/full/2023_122000_484000_RGB_hrl.tif"
resolution = 0.08

image_data = ImageData(full_image_path)

annotations_folder_path = os.path.join(
    Folders.CROPPED_ANNOTS.value, image_data.base_name
)
rgb_folder_path = os.path.join(Folders.CROPPED_IMAGES.value, image_data.base_name)
chm_folder_path = os.path.join(
    Folders.CHM.value,
    f"{round(resolution*100)}cm",
    "unfiltered",
    "cropped",
    image_data.coord_name,
)

train_dataset = TreeDataset(
    annotations_folder_path=annotations_folder_path,
    rgb_folder_path=rgb_folder_path,
    chm_folder_path=chm_folder_path,
    labels_to_index=class_indices,
    # transform_spatial=transform_spatial,
    # transform_pixel=transform_pixel,
    transform_spatial=None,
    transform_pixel=None,
)

val_dataset = TreeDataset(
    annotations_folder_path=annotations_folder_path,
    rgb_folder_path=rgb_folder_path,
    chm_folder_path=chm_folder_path,
    labels_to_index=class_indices,
    transform_spatial=None,
    transform_pixel=None,
)

# Training parameters

lr = 1
epochs = 10

batch_size = 4
num_workers = 4

train_and_validate(
    model=model,
    train_dataset=train_dataset,
    val_dataset=val_dataset,
    lr=lr,
    epochs=epochs,
    batch_size=batch_size,
    num_workers=num_workers,
    device=device,
)

Adjusting learning rate of group 0 to 7.0711e-01.


/home/alexandre/miniforge3/envs/tree-segment/lib/python3.12/site-packages/osgeo/gdal.py:312: FutureWarning: Neither gdal.UseExceptions() nor gdal.DontUseExceptions() has been explicitly called. In GDAL 4.0, exceptions will be enabled by default.
  warnings.warn(


Adjusting learning rate of group 0 to 5.7735e-01.


Adjusting learning rate of group 0 to 5.0000e-01.


Adjusting learning rate of group 0 to 4.4721e-01.


...

Adjusting learning rate of group 0 to 3.7796e-01.


Adjusting learning rate of group 0 to 3.5355e-01.


Adjusting learning rate of group 0 to 3.3333e-01.


Adjusting learning rate of group 0 to 3.1623e-01.


Adjusting learning rate of group 0 to 3.0151e-01.


In [27]:
# def print_device_of_layers(model):
#     for name, module in model.named_modules():
#         try:
#             device = next(iter(module.parameters())).device
#             # if device != torch.device("cuda:0"):
#             #     print(f"{name}: {device}")
#             print(f"{name}: {device}")
#         except StopIteration:
#             pass  # Skip modules without parameters


# print_device_of_layers(model)

for name, module in model.named_modules():
    try:
        print(f"{name}: {next(iter(module.parameters()))[0]}")
    except StopIteration:
        pass  # Skip modules without parameters

: tensor([[[4.2544, 4.1491, 4.3862],
         [4.4382, 4.1582, 4.0894],
         [4.1615, 4.1980, 4.3474]],

        [[4.2457, 4.1861, 4.1849],
         [4.4111, 4.1053, 4.2496],
         [4.3241, 4.3012, 4.4017]],

        [[4.2455, 4.3304, 4.2873],
         [4.2495, 4.3494, 4.0651],
         [4.3386, 4.1993, 4.1493]]], device='cuda:0', grad_fn=<SelectBackward0>)
amfnet: tensor([[[4.2544, 4.1491, 4.3862],
         [4.4382, 4.1582, 4.0894],
         [4.1615, 4.1980, 4.3474]],

        [[4.2457, 4.1861, 4.1849],
         [4.4111, 4.1053, 4.2496],
         [4.3241, 4.3012, 4.4017]],

        [[4.2455, 4.3304, 4.2873],
         [4.2495, 4.3494, 4.0651],
         [4.3386, 4.1993, 4.1493]]], device='cuda:0', grad_fn=<SelectBackward0>)
amfnet.yolo_backbone_left: tensor([[[4.2544, 4.1491, 4.3862],
         [4.4382, 4.1582, 4.0894],
         [4.1615, 4.1980, 4.3474]],

        [[4.2457, 4.1861, 4.1849],
         [4.4111, 4.1053, 4.2496],
         [4.3241, 4.3012, 4.4017]],

        [[4.2455, 4

### Transformation tests

In [ ]:
# crop_size = 640
# distort_steps = 30
# distort_limit = 0.2

# transform = A.Compose(
#     [
#         A.RandomCrop(width=crop_size, height=crop_size, p=1.0),
#     ],
# )

# transform_2 = A.Compose(
#     [
#         A.RandomCrop(width=crop_size, height=crop_size, p=1.0),
#         A.Sharpen(p=0.25),
#         A.RingingOvershoot(p=0.5),
#         A.RandomGamma(p=1.0),
#         A.GaussianBlur(p=0.5),
#         A.GaussNoise(p=0.5),
#         A.FancyPCA(alpha=1.0, p=0.5),
#         A.Emboss(p=0.5),
#         A.Blur(p=0.5),
#         A.RandomBrightnessContrast(p=1.0),
#         A.ChannelDropout(p=0.25),
#         A.CLAHE(clip_limit=2.0, p=0.25),
#         A.Normalize(p=1.0),
#     ],
# )

# image_path = "../data/images_cropped/2023_122000_484000_RGB_hrl/2023_122000_484000_RGB_hrl_2_3.tif"
# # image_path = "../data/flat-design-abstract-outline-background_23-2150616456.jpg"

# t = time()

# image = cv2.imread(image_path)

# print(f"Time elapsed: {round(time() - t, 3)}")

# number_tests = 1

# t = time()

# for i in range(number_tests):
#     r = random.random()
#     random.seed(r)
#     transformed_image = transform(image=image)["image"]
#     random.seed(r)
#     transformed_image_2 = transform_2(image=image)["image"]

#     output_name = f"Augmentation_test_{i}.tif"
#     output_path = os.path.join(OUTPUT_DIR, output_name)
#     cv2.imwrite(output_path, transformed_image)

#     output_name = f"Augmentation_test_{i}_2.tif"
#     output_path = os.path.join(OUTPUT_DIR, output_name)
#     cv2.imwrite(output_path, transformed_image_2)

# print(f"Time elapsed: {round(time() - t, 3)}")